# **Maestría en Inteligencia Artificial Aplicada**

## **Curso:** Análisis de grandes volúmenes de datos

## **Tecnológico de Monterrey**

## **Actividad 4:**  Métricas de calidad de resultados

## **Nombre:** Oscar Luis Guadarrama Jiménez
## **Matricula:** A01796245

## **Librerías utilizadas**

A continuación, se importan las librerías necesarias para el procesamiento, modelado y evaluación de datos usando PySpark.


In [1]:
# Sesión de Spark
from pyspark.sql import SparkSession

# Transformaciones sobre columnas
from pyspark.sql.functions import col, when, isnan, count,round,regexp_replace, expr, lit, percentile_approx, first,sum,skewness, log1p,round
from pyspark.sql import functions as F
from pyspark.sql.functions import col as spark_col
from pyspark.sql.types import IntegerType

# Preprocesamiento
from pyspark.ml.feature import StringIndexer, VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline

# Modelos supervisados
from pyspark.ml.classification import RandomForestClassifier

# Modelos no supervisados
from pyspark.ml.clustering import KMeans

# Evaluación
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, ClusteringEvaluator,BinaryClassificationEvaluator

#Visualización y análisis con Pandas y Matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
spark = SparkSession.builder \
    .appName("Entrenamiento Optimizado") \
    .master("local[*]") \
    .config("spark.driver.memory", "48g") \
    .config("spark.executor.memory", "48g") \
    .config("spark.sql.shuffle.partitions", "64") \
    .config("spark.default.parallelism", "64") \
    .getOrCreate()

In [3]:
spark


## **Cargar el dataset**

In [4]:
# Cargar el dataset
df = spark.read.option("header", True).option("inferSchema", True).csv("./loan.csv")

# Mostrar las primeras filas
df.show(5)

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-------

# **1. Construcción de la muestra M**

Esta sección tiene como objetivo construir una muestra representativa `M` a partir de la población original `P`, aplicando un conjunto riguroso de transformaciones y filtrados para garantizar su calidad, homogeneidad y utilidad en procesos de entrenamiento y evaluación posteriores.

---

## **1.1 Exploración inicial y análisis de estructura**

Se inició con la visualización de las primeras filas del dataset y se verificó que contenía un total de **145 columnas**. A continuación, se realizó un mapeo de los tipos esperados por columna y se aplicó una conversión automática para corregir tipos mal definidos, incluyendo el tratamiento de valores numéricos que venían como cadenas (`string` → `double`).

---

## **1.2 Análisis de valores nulos y limpieza preliminar**

Se ejecutó un conteo de valores nulos por columna. Con base en esto, se implementaron las siguientes reglas:

- Se eliminaron todas las columnas con más del **20% de valores nulos**.
- Se imputaron valores faltantes en variables numéricas utilizando la **mediana**.
- Se imputaron valores faltantes en variables categóricas utilizando la **moda**.

Después de este proceso, se redujo la cantidad de columnas a aquellas con mayor integridad estructural.

---

## **1.3 Eliminación de columnas irrelevantes o problemáticas**

Se aplicaron múltiples filtros para eliminar columnas que podrían introducir ruido o distorsionar el análisis:

- **Texto libre o cardinalidad alta:** columnas con más de 10,000 valores únicos (`emp_title`, `title`).
- **Redundancia lineal:** columnas con una correlación mayor a `0.999` entre sí.
- **Baja varianza en variables categóricas:** columnas donde un único valor aparece en más del 70% de las observaciones.
- **Dominancia de ceros en variables numéricas:** columnas con más del 70% de sus registros en cero.
- **Fuga de información:** columnas como `issue_d`, `last_pymnt_d` y `last_credit_pull_d` fueron eliminadas para evitar sesgos durante el entrenamiento.

También se eliminaron otras columnas irrelevantes según criterios semánticos y de contexto.

---

## **1.4 Reducción de categorías y redefinición de variables clave**

Con el objetivo de simplificar el análisis:

- Se agruparon las categorías poco frecuentes en `home_ownership`, `purpose` y `loan_status`.
- Se definió una nueva variable binaria `loan_status` codificada como:
  - **1 = riesgo_alto**
  - **0 = No_riesgo**

Estas transformaciones facilitarán la aplicación de modelos supervisados de clasificación.

---

## **1.5 Análisis de outliers y escalamiento logarítmico**

Se identificaron valores atípicos extremos en variables numéricas mediante el criterio de IQR. Se eliminaron aquellos registros fuera de los límites superiores e inferiores calculados. Además, se aplicó una **transformación logarítmica** a variables con alta asimetría (`skewness`) y gran presencia de outliers, mejorando su distribución.

---

## **1.6 Selección de variables de caracterización**

Se definieron tres variables clave para la estratificación del dataset, considerando su relación con el problema de análisis:

- `purpose`
- `home_ownership`
- `verification_status`

Estas variables se utilizarán para agrupar el dataset en subconjuntos homogéneos, denominados `Mi`.

---

## **1.7 Construcción de la muestra representativa M**

Se evaluaron todas las combinaciones posibles entre las variables de caracterización y se seleccionaron únicamente aquellas que contaban con **más de 60,000 registros**, asegurando robustez en cada subconjunto.

Posteriormente, sobre cada combinación válida se aplicó un **muestreo aleatorio del 20%** sin reemplazo (`fraction=0.2`), y se construyó la muestra `M` como la **unión de todos los subconjuntos `Mi`**.

Finalmente, se verificó la distribución de combinaciones en la muestra `M` para confirmar que se preservaran las proporciones representativas de la población original.

---

**Resultado final:** la muestra `M` quedó compuesta por **336,021 registros**, agrupados en 10 combinaciones robustas de propósito, tipo de vivienda y estado de verificación. Esta muestra servirá como base para la etapa de entrenamiento y evaluación de modelos.


In [5]:
# Mostrar las primeras filas
df.show(5)

+----+---------+---------+-----------+---------------+----------+--------+-----------+-----+---------+--------------+----------+--------------+----------+-------------------+--------+-----------+----------+----+----+------------------+------------------+--------+----------+-----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+-------

In [6]:
#Número total de columnas al inicio
print(f"Número total de columnas en el dataset original: {len(df.columns)}")

Número total de columnas en el dataset original: 145


In [7]:
tipos_esperados = {
    "loan_amnt": "int",
    "funded_amnt": "int",
    "funded_amnt_inv": "double",
    "term": "string",
    "int_rate": "double",
    "installment": "double",
    "grade": "string",
    "sub_grade": "string",
    "emp_title": "string",
    "emp_length": "string",
    "home_ownership": "string",
    "annual_inc": "double",
    "verification_status": "string",
    "issue_d": "string",
    "loan_status": "string",
    "pymnt_plan": "string",
    "purpose": "string",
    "title": "string",
    "zip_code": "string",
    "addr_state": "string",
    "dti": "double",
    "delinq_2yrs": "int",
    "earliest_cr_line": "int",
    "inq_last_6mths": "int",
    "open_acc": "int",
    "pub_rec": "int",
    "revol_bal": "int",
    "revol_util": "double",
    "total_acc": "int",
    "initial_list_status": "string",
    "out_prncp": "int",
    "out_prncp_inv": "double",
    "total_pymnt": "double",
    "total_pymnt_inv": "double",
    "total_rec_prncp": "double",
    "total_rec_int": "double",
    "total_rec_late_fee": "double",
    "recoveries": "double",
    "collection_recovery_fee": "double",
    "last_pymnt_d": "string",
    "last_pymnt_amnt": "double",
    "last_credit_pull_d": "string",
    "collections_12_mths_ex_med": "int",
    "policy_code": "int",
    "application_type": "string",
    "acc_now_delinq": "int",
    "tot_coll_amt": "int",
    "tot_cur_bal": "int",
    "total_rev_hi_lim": "int",
    "acc_open_past_24mths": "int",
    "avg_cur_bal": "double",
    "bc_open_to_buy": "int",
    "bc_util": "double",
    "chargeoff_within_12_mths": "double",
    "delinq_amnt": "int",
    "mo_sin_old_il_acct": "int",
    "mo_sin_old_rev_tl_op": "int",
    "mo_sin_rcnt_rev_tl_op": "int",
    "mo_sin_rcnt_tl": "int",
    "mort_acc": "int",
    "mths_since_recent_bc": "int",
    "mths_since_recent_inq": "int",
    "num_accts_ever_120_pd": "int",
    "num_actv_bc_tl": "int",
    "num_actv_rev_tl": "int",
    "num_bc_sats": "int",
    "num_bc_tl": "int",
    "num_il_tl": "int",
    "num_op_rev_tl": "int",
    "num_rev_accts": "int",
    "num_rev_tl_bal_gt_0": "int",
    "num_sats": "int",
    "num_tl_120dpd_2m": "int",
    "num_tl_30dpd": "int",
    "num_tl_90g_dpd_24m": "int",
    "num_tl_op_past_12m": "int",
    "pct_tl_nvr_dlq": "double",
    "percent_bc_gt_75": "double",
    "pub_rec_bankruptcies": "int",
    "tax_liens": "int",
    "tot_hi_cred_lim": "int",
    "total_bal_ex_mort": "int",
    "total_bc_limit": "int",
    "total_il_high_credit_limit": "int",
    "hardship_flag": "string",
    "disbursement_method": "string",
    "debt_settlement_flag": "string",
}


In [8]:
# Corrección automática de tipos según tipos_esperados
for columna, tipo_esperado in tipos_esperados.items():
    if columna in dict(df.dtypes):
        tipo_actual = dict(df.dtypes)[columna]
        if tipo_actual != tipo_esperado:
            print(f"Corrigiendo: {columna} de {tipo_actual} a {tipo_esperado}")
            
            # Limpieza de strings si es necesario para convertir a double
            if tipo_esperado == "double" and tipo_actual == "string":
                df = df.withColumn(columna, regexp_replace(col(columna), "[,%$]", "").cast("double"))
            else:
                df = df.withColumn(columna, col(columna).cast(tipo_esperado))


Corrigiendo: annual_inc de string a double
Corrigiendo: dti de string a double
Corrigiendo: delinq_2yrs de string a int
Corrigiendo: earliest_cr_line de string a int
Corrigiendo: inq_last_6mths de string a int
Corrigiendo: open_acc de string a int
Corrigiendo: pub_rec de string a int
Corrigiendo: revol_bal de string a int
Corrigiendo: revol_util de string a double
Corrigiendo: total_acc de string a int
Corrigiendo: out_prncp de string a int
Corrigiendo: out_prncp_inv de string a double
Corrigiendo: total_pymnt de string a double
Corrigiendo: total_pymnt_inv de string a double
Corrigiendo: total_rec_prncp de string a double
Corrigiendo: total_rec_int de string a double
Corrigiendo: total_rec_late_fee de string a double
Corrigiendo: recoveries de string a double
Corrigiendo: collection_recovery_fee de string a double
Corrigiendo: last_pymnt_amnt de string a double
Corrigiendo: collections_12_mths_ex_med de string a int
Corrigiendo: policy_code de string a int
Corrigiendo: acc_now_delinq 

In [9]:
# Ver cuántos valores nulos hay por columna 
print("Valores nulos por columna:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show(truncate=False)


Valores nulos por columna:
+-------+---------+---------+-----------+---------------+----+--------+-----------+-----+---------+---------+----------+--------------+----------+-------------------+-------+-----------+----------+-------+-------+-------+-----+--------+----------+----+-----------+----------------+--------------+----------------------+----------------------+--------+-------+---------+----------+---------+-------------------+---------+-------------+-----------+---------------+---------------+-------------+------------------+----------+-----------------------+------------+---------------+------------+------------------+--------------------------+---------------------------+-----------+----------------+----------------+---------+-------------------------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------------+------------+-------+-----------+-----------+----------+--------+----------------+------+-----------+------------+--------

In [10]:
#ELminar columans con el 20% de vlaores nuloso 
# Calcular total de registros
total_rows = df.count()

# Calcular porcentaje de nulos por columna y quedarte con las que tienen menos del 20%
col_porcentaje_nulos = [
   (sum(when((col(c).isNull()) | (col(c) == ''), 1).otherwise(0)) / total_rows).alias(c)
    for c in df.columns
]

# Generar DataFrame con porcentaje de nulos
porcentaje_nulos_df = df.select(col_porcentaje_nulos)
porcentaje_nulos = porcentaje_nulos_df.collect()[0].asDict()

# Filtrar columnas que tienen menos de 20% de nulos
columnas_validas = [k for k, v in porcentaje_nulos.items() if v < 0.2]

# Seleccionar solo las columnas válidas
df = df.select(*columnas_validas)

print(f"Columnas seleccionadas con <20% de nulos: {len(columnas_validas)}")


Columnas seleccionadas con <20% de nulos: 86


In [11]:
# Columnas con más de 10,000 valores únicos de tipo string (texto libre)
columnas_texto_libre = []

for c in df.columns:
    if dict(df.dtypes)[c] == "string" and df.select(c).distinct().count() > 10000:
        columnas_texto_libre.append(c)

print("Columnas con texto libre o muchas categorías únicas:")
print(columnas_texto_libre)


Columnas con texto libre o muchas categorías únicas:
['emp_title', 'title']


In [12]:
df = df.drop(*columnas_texto_libre)

print(f"Columnas eliminadas por texto libre o alta cardinalidad: {len(columnas_texto_libre)}")
print(f"Total de columnas restantes: {len(df.columns)}")


Columnas eliminadas por texto libre o alta cardinalidad: 2
Total de columnas restantes: 84


In [13]:
# Seleccionar solo columnas numéricas del DataFrame
columnas_numericas = [c for c, t in df.dtypes if t in ("double", "int")]

# Convertimos a Pandas una muestra del dataframe para no saturar memoria
df_sample_pd = df.select(columnas_numericas).sample(False, 0.10, seed=1).toPandas()

# Calculamos matriz de correlación absoluta
cor_matrix = df_sample_pd.corr().abs()

columnas_correladas = set()
umbral = 0.999

for i in range(len(cor_matrix.columns)):
    for j in range(i + 1, len(cor_matrix.columns)):
        col1 = cor_matrix.columns[i]
        col2 = cor_matrix.columns[j]
        if cor_matrix.iloc[i, j] > umbral:
            print(f"Alta correlación entre: {col1} y {col2} (r = {cor_matrix.iloc[i, j]:.4f})")
            columnas_correladas.add(col2)  # Podrías quedarte con el primero, y marcar el segundo para eliminar

# Resultado: columnas candidatas a eliminar por redundancia lineal
columnas_correladas = list(columnas_correladas)


Alta correlación entre: loan_amnt y funded_amnt (r = 0.9997)
Alta correlación entre: loan_amnt y funded_amnt_inv (r = 0.9991)
Alta correlación entre: funded_amnt y funded_amnt_inv (r = 0.9994)
Alta correlación entre: out_prncp y out_prncp_inv (r = 1.0000)
Alta correlación entre: total_pymnt y total_pymnt_inv (r = 0.9994)


In [14]:
# Eliminamos columnas por redundancia lineal
df = df.drop(*list(columnas_correladas))
print(f"Columnas eliminadas por redundancia lineal: {len(columnas_correladas)}")
print(f"Total de columnas restantes: {len(df.columns)}")

Columnas eliminadas por redundancia lineal: 4
Total de columnas restantes: 80


In [15]:
# Detectamos columnas categóricas automáticamente (tipo string o categoricas enteras conocidas)
columnas_categoricas = [c for c, t in df.dtypes if t == "string" or "emp_length" in c]
print(f"Columnas categóricas detectadas: {len(columnas_categoricas)}")
print(columnas_categoricas)


Columnas categóricas detectadas: 19
['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'zip_code', 'addr_state', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type', 'hardship_flag', 'disbursement_method', 'debt_settlement_flag']


In [16]:
# Eliminamos columnas por  baja varianza
columnas_baja_varianza = []
umbral = 0.70
total_rows = df.count()

for c in columnas_categoricas:
    top_valor = df.groupBy(c).count().orderBy("count", ascending=False).first()
    if top_valor and (top_valor["count"] / total_rows) >= umbral:
        print(f"{c}: {top_valor['count'] / total_rows:.2%} del dataset tiene '{top_valor[c]}'")
        columnas_baja_varianza.append(c)



term: 71.21% del dataset tiene ' 36 months'
pymnt_plan: 99.97% del dataset tiene 'n'
application_type: 94.65% del dataset tiene 'Individual'
hardship_flag: 99.95% del dataset tiene 'N'
disbursement_method: 96.53% del dataset tiene 'Cash'
debt_settlement_flag: 98.53% del dataset tiene 'N'


In [17]:
df = df.drop(*columnas_baja_varianza)
print(f"Columnas eliminadas por baja varianza: {len(columnas_baja_varianza)}")
print(f"Total de columnas restantes: {len(df.columns)}")

Columnas eliminadas por baja varianza: 6
Total de columnas restantes: 74


In [18]:
# Eliminaipn por fuga de informacion
columnas_fuga_info = [
    "issue_d",             # Fecha del préstamo
    "last_pymnt_d",        # Fecha del último pago
    "last_credit_pull_d",  # Fecha del último análisis de crédito
]
df = df.drop(*columnas_fuga_info)
print(f"Columnas eliminadas por fuga de informacion: {len(columnas_fuga_info)}")
print(f"Total de columnas restantes: {len(df.columns)}")



Columnas eliminadas por fuga de informacion: 3
Total de columnas restantes: 71


In [19]:
# Detectar columnas numéricas 
columnas_numericas = [c for c, t in df.dtypes if t in ["double", "int"]]

# Eliminamos columnas por  baja varianza

umbral = 0.70
total_rows = df.count()
columnas_valor_dominante = []

for c in columnas_numericas:
    valor_mas_frecuente = (
        df.groupBy(c)
        .count()
        .orderBy("count", ascending=False)
        .first()
    )

    if valor_mas_frecuente:
        valor = valor_mas_frecuente[0]  # el valor dominante
        cuenta = valor_mas_frecuente["count"]
        porcentaje = cuenta / total_rows

        if porcentaje >= umbral:
            print(f"{c}: {porcentaje:.2%} de los valores son '{valor}'")
            columnas_valor_dominante.append(c)


delinq_2yrs: 81.34% de los valores son '0'
pub_rec: 84.16% de los valores son '0'
total_rec_late_fee: 96.25% de los valores son '0.0'
recoveries: 92.14% de los valores son '0.0'
collection_recovery_fee: 92.52% de los valores son '0.0'
collections_12_mths_ex_med: 98.33% de los valores son '0'
policy_code: 99.99% de los valores son '1'
acc_now_delinq: 99.60% de los valores son '0'
tot_coll_amt: 82.11% de los valores son '0'
chargeoff_within_12_mths: 99.22% de los valores son '0.0'
delinq_amnt: 99.67% de los valores son '0'
num_accts_ever_120_pd: 74.64% de los valores son '0'
num_tl_120dpd_2m: 93.15% de los valores son '0'
num_tl_30dpd: 96.63% de los valores son '0'
num_tl_90g_dpd_24m: 91.70% de los valores son '0'
pub_rec_bankruptcies: 87.90% de los valores son '0'
tax_liens: 97.13% de los valores son '0'


In [20]:
df = df.drop(*columnas_valor_dominante)
print(f"Columnas eliminadas por tener >70% ceros: {len(columnas_valor_dominante)}")
print(f"Total de columnas restantes: {len(df.columns)}")

Columnas eliminadas por tener >70% ceros: 17
Total de columnas restantes: 54


In [21]:
# Lista de columnas irrelevantes

columnas_irrelevantes = [

     "zip_code", # zip_code: Dato personal, alta cardinalidad, difícil de codificar.
      "addr_state", # addr_state: Redundante con home_ownership; baja capacidad predictiva.
      "sub_grade",# sub_grade: Desglose innecesario de grade (A1, A2...), ya se usa grade.
      "initial_list_status",    # initial_list_status: metadato técnico sin valor predictivo
 ]
# Eliminamos del DataFrame
df = df.drop(*[c for c in columnas_irrelevantes if c in df.columns])

print(f"Columnas eliminadas por irrelevancia justificada: {len(columnas_irrelevantes)}")
print(f"Columnas finales restantes: {len(df.columns)}")


Columnas eliminadas por irrelevancia justificada: 4
Columnas finales restantes: 50


In [22]:
# Lista de columnas que se eliminan por definición técnica o redundancia
columnas_a_eliminar_por_definicion = [
    "bc_open_to_buy",          # Redundante con bc_util (porcentaje de uso vs. crédito disponible)
    "mo_sin_old_il_acct",      # Tiempo desde cuenta a plazo más antigua; técnica, poco explicativa
    "mo_sin_old_rev_tl_op",    # Tiempo desde cuenta revolvente antigua; similar a la anterior
    "mo_sin_rcnt_rev_tl_op",   # Tiempo desde última línea revolvente; se solapa con mo_sin_rcnt_tl
    "mo_sin_rcnt_tl",          # Tiempo desde última cuenta abierta; técnica, poco interpretable
    "num_bc_tl",               # Total de tarjetas de crédito abiertas históricamente; poco diferenciador
    "num_op_rev_tl",           # Líneas revolventes abiertas; similar a num_rev_tl_bal_gt_0
    "num_rev_accts",           # Cuentas revolventes históricas; métrica muy general
    "total_bc_limit"           # Límite total en tarjetas; ya considerado en tot_hi_cred_lim
]

# Eliminar solo si existen en el DataFrame
df = df.drop(*[c for c in columnas_a_eliminar_por_definicion if c in df.columns])

# Reportar cuántas columnas se eliminaron
print(f"Columnas eliminadas por definición: {len(columnas_a_eliminar_por_definicion)}")
print(f"Columnas finales restantes: {len(df.columns)}")


Columnas eliminadas por definición: 9
Columnas finales restantes: 41


In [23]:
columnas_categoricas = [c for c, t in df.dtypes if t == "string"]

for col_name in columnas_categoricas:
    total_distintos = df.select(col_name).distinct().count()
    df.groupBy(col_name).count().orderBy("count", ascending=False).show()
    

+-----+------+
|grade| count|
+-----+------+
|    B|663557|
|    C|650053|
|    A|433027|
|    D|324424|
|    E|135639|
|    F| 41800|
|    G| 12168|
+-----+------+

+----------+------+
|emp_length| count|
+----------+------+
| 10+ years|748005|
|   2 years|203676|
|  < 1 year|189988|
|   3 years|180753|
|    1 year|148403|
|       n/a|146907|
|   5 years|139698|
|   4 years|136605|
|   6 years|102628|
|   7 years| 92695|
|   8 years| 91914|
|   9 years| 79395|
| reactors"|     1|
+----------+------+

+--------------+-------+
|home_ownership|  count|
+--------------+-------+
|      MORTGAGE|1111449|
|          RENT| 894929|
|           OWN| 253057|
|           ANY|    996|
|         OTHER|    182|
|          NONE|     54|
|       2 years|      1|
+--------------+-------+

+-------------------+------+
|verification_status| count|
+-------------------+------+
|    Source Verified|886230|
|       Not Verified|744806|
|           Verified|629631|
|              38000|     1|
+-------------

In [24]:
# Detectar columnas numéricas
columnas_numericas = [c for c, t in df.dtypes if t in ["int", "double"]]

# Usar describe para estadísticas generales
df.select(columnas_numericas).describe().show()


+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+-----------------+------------------+------------------+--------------------+------------------+------------------+------------------+--------------------+---------------------+------------------+-----------------+------------------+-----------------+-------------------+------------------+------------------+----------------+-----------------+------------------+------------------+--------------------------+
|summary|         loan_amnt|          int_rate|       installment|        annual_inc|               dti|    inq_last_6mths|          open_acc|         revol_bal|        revol_util|         total_acc|        out_prncp|      total_pymnt|  total_rec_prncp|     total_rec_int|  last_pymnt_amnt|       tot_cur_bal|  total_r

In [25]:
categorias_frecuentes = ['MORTGAGE','RENT','OWN']
# Reducir categories en 'home_ownership'
df = df.withColumn(
    "home_ownership",
    when(col("home_ownership").isin(categorias_frecuentes), col("home_ownership"))
    .otherwise("OTHER")
)
# Reducir categories en 'purpose'
df = df.withColumn("purpose", when(col("purpose").isin(
    "debt_consolidation", "credit_card"), "debt_related")
    .when(col("purpose").isin("home_improvement", "major_purchase"), "large_expense")
    .when(col("purpose").isin("small_business", "car", "vacation", "house"), "personal_use")
    .when(col("purpose").isin("medical", "wedding", "moving"), "life_event")
    .otherwise("other"))

# Reducir categories en 'loan_status'
df = df.withColumn("loan_status",
    when(col("loan_status").isin("Fully Paid", "Current"), "No_riesgo")
    .when(col("loan_status").isin(
        "Late (16-30 days)", "In Grace Period",
        "Charged Off", "Default", "Late (31-120 days)",
        "Does not meet the credit policy. Status:Fully Paid",
        "Does not meet the credit policy. Status:Charged Off",
        "Oct-2015", None), "riesgo")
)


In [26]:
columnas_categoricas = [c for c, t in df.dtypes if t == "string"]

for col_name in columnas_categoricas:
    total_distintos = df.select(col_name).distinct().count()
    df.groupBy(col_name).count().orderBy("count", ascending=False).show()
    

+-----+------+
|grade| count|
+-----+------+
|    B|663557|
|    C|650053|
|    A|433027|
|    D|324424|
|    E|135639|
|    F| 41800|
|    G| 12168|
+-----+------+

+----------+------+
|emp_length| count|
+----------+------+
| 10+ years|748005|
|   2 years|203676|
|  < 1 year|189988|
|   3 years|180753|
|    1 year|148403|
|       n/a|146907|
|   5 years|139698|
|   4 years|136605|
|   6 years|102628|
|   7 years| 92695|
|   8 years| 91914|
|   9 years| 79395|
| reactors"|     1|
+----------+------+

+--------------+-------+
|home_ownership|  count|
+--------------+-------+
|      MORTGAGE|1111449|
|          RENT| 894929|
|           OWN| 253057|
|         OTHER|   1233|
+--------------+-------+

+-------------------+------+
|verification_status| count|
+-------------------+------+
|    Source Verified|886230|
|       Not Verified|744806|
|           Verified|629631|
|              38000|     1|
+-------------------+------+

+-----------+-------+
|loan_status|  count|
+-----------+--

In [27]:
# Ver cuántos valores nulos hay por columna 
print("Valores nulos por columna:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show(truncate=False)

Valores nulos por columna:
+---------+--------+-----------+-----+----------+--------------+----------+-------------------+-----------+-------+----+--------------+--------+---------+----------+---------+---------+-----------+---------------+-------------+---------------+-----------+----------------+--------------------+-----------+-------+--------+--------------------+---------------------+--------------+---------------+-----------+---------+-------------------+--------+------------------+--------------+----------------+---------------+-----------------+--------------------------+
|loan_amnt|int_rate|installment|grade|emp_length|home_ownership|annual_inc|verification_status|loan_status|purpose|dti |inq_last_6mths|open_acc|revol_bal|revol_util|total_acc|out_prncp|total_pymnt|total_rec_prncp|total_rec_int|last_pymnt_amnt|tot_cur_bal|total_rev_hi_lim|acc_open_past_24mths|avg_cur_bal|bc_util|mort_acc|mths_since_recent_bc|mths_since_recent_inq|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_i

In [28]:
# Imputación de variables numéricas con MEDIANA
for col_name in columnas_numericas:
    mediana = df.approxQuantile(col_name, [0.5], 0.01)[0]
    df = df.withColumn(
        col_name,
        when(col(col_name).isNull(), lit(mediana)).otherwise(col(col_name))
    )

# Imputación de variables categóricas con MODO (valor más frecuente)
for col_name in columnas_categoricas:
    modo = (df.groupBy(col_name)
                     .count()
                     .orderBy(F.desc("count"))
                     .first()[0])
    df = df.withColumn(
        col_name,
        when(col(col_name).isNull(), lit(modo)).otherwise(col(col_name))
    )

In [29]:
# Ver cuántos valores nulos hay por columna 
print("Valores nulos por columna:")
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show(truncate=False)

Valores nulos por columna:
+---------+--------+-----------+-----+----------+--------------+----------+-------------------+-----------+-------+---+--------------+--------+---------+----------+---------+---------+-----------+---------------+-------------+---------------+-----------+----------------+--------------------+-----------+-------+--------+--------------------+---------------------+--------------+---------------+-----------+---------+-------------------+--------+------------------+--------------+----------------+---------------+-----------------+--------------------------+
|loan_amnt|int_rate|installment|grade|emp_length|home_ownership|annual_inc|verification_status|loan_status|purpose|dti|inq_last_6mths|open_acc|revol_bal|revol_util|total_acc|out_prncp|total_pymnt|total_rec_prncp|total_rec_int|last_pymnt_amnt|tot_cur_bal|total_rev_hi_lim|acc_open_past_24mths|avg_cur_bal|bc_util|mort_acc|mths_since_recent_bc|mths_since_recent_inq|num_actv_bc_tl|num_actv_rev_tl|num_bc_sats|num_il_

In [30]:
# Obtener skewness
skew_data = df.select([skewness(c).alias(c) for c in columnas_numericas]).toPandas().T
skew_data.columns = ['skewness']
skew_data['feature'] = skew_data.index


In [31]:
outlier_info = []

for col_name in columnas_numericas:
    q1, q3 = df.approxQuantile(col_name, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    lower = q1 - 1.5 * iqr
    upper = q3 + 1.5 * iqr
    outlier_count = df.filter((col(col_name) < lower) | (col(col_name) > upper)).count()
    outlier_info.append((col_name, outlier_count))

import pandas as pd
outlier_df = pd.DataFrame(outlier_info, columns=["feature", "outlier_count"])


In [32]:
merged = skew_data.merge(outlier_df, on="feature")
merged["decision"] = merged.apply(lambda row:
    "log" if abs(row["skewness"]) > 1.0 and row["outlier_count"] > 10000 else
    "delete" if row["outlier_count"] > 0 and abs(row["skewness"]) < 1.0 else
    "keep", axis=1
)

merged.sort_values("decision", ascending=False)


,skewness,feature,outlier_count,decision
17,1.427695,acc_open_past_24mths,48076,log
16,32.998927,total_rev_hi_lim,144364,log
33,4.291340,total_bal_ex_mort,144886,log
32,3.899698,tot_hi_cred_lim,90543,log
30,-2.322728,pct_tl_nvr_dlq,165439,log
29,1.531233,num_tl_op_past_12m,55466,log
28,1.340079,num_sats,83072,log
27,1.508392,num_rev_tl_bal_gt_0,59829,log
26,2.156230,num_il_tl,100049,log
25,1.787362,num_bc_sats,106663,log


In [33]:
cols_log = merged[merged["decision"] == "log"]["feature"].tolist()
cols_delete = merged[merged["decision"] == "delete"]["feature"].tolist()
cols_keep = merged[merged["decision"] == "keep"]["feature"].tolist()

print("Columnas para log:", cols_log)
print("Columnas para eliminar outliers:", cols_delete)
print("Columnas para conservar:", cols_keep)

Columnas para log: ['installment', 'annual_inc', 'dti', 'inq_last_6mths', 'open_acc', 'revol_bal', 'total_acc', 'out_prncp', 'total_pymnt', 'total_rec_prncp', 'total_rec_int', 'last_pymnt_amnt', 'tot_cur_bal', 'total_rev_hi_lim', 'acc_open_past_24mths', 'avg_cur_bal', 'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_inq', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_il_tl', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_il_high_credit_limit']
Columnas para eliminar outliers: ['loan_amnt', 'int_rate', 'bc_util']
Columnas para conservar: ['revol_util', 'percent_bc_gt_75']


In [34]:

py_round = __builtins__.round 

def eliminar_outliers_spark(df, columnas):
    for nombre_col in columnas:
        # Calcular Q1 y Q3
        q1, q3 = df.approxQuantile(nombre_col, [0.25, 0.75], 0.01)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr

        # Filtrar fuera del IQR
        df = df.filter(
            (spark_col(nombre_col) >= limite_inferior) & (spark_col(nombre_col) <= limite_superior)
        )
        
    return df


In [35]:

df_v1 = eliminar_outliers_spark(df, cols_delete)

In [36]:

for c in cols_log:
    df_v1 = df_v1.withColumn(f"{c}", log1p(col(c)))


In [37]:
df_v1.show(5)

+---------+--------+-----------------+-----+----------+--------------+------------------+-------------------+-----------+------------+------------------+------------------+------------------+-----------------+----------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+------------------+--------------------+------------------+-------+------------------+--------------------+---------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+----------------+------------------+------------------+--------------------------+
|loan_amnt|int_rate|      installment|grade|emp_length|home_ownership|        annual_inc|verification_status|loan_status|     purpose|               dti|    inq_last_6mths|          open_acc|        revol_bal|revol_util|         total_acc|         out_prncp|       total_pymnt|  total_

In [38]:
total = df_v1.count()

combinaciones = df_v1.groupBy('purpose', 'home_ownership','verification_status') \
    .count() \
    .withColumnRenamed("count", "frecuencia") \
    .withColumn("probabilidad (%)", round((col("frecuencia") / total) * 100, 2)) \
    .orderBy("frecuencia", ascending=False)

print("Combinaciones posibles:", combinaciones.count())
combinaciones.show(truncate=False)

Combinaciones posibles: 61
+-------------+--------------+-------------------+----------+----------------+
|purpose      |home_ownership|verification_status|frecuencia|probabilidad (%)|
+-------------+--------------+-------------------+----------+----------------+
|debt_related |MORTGAGE      |Source Verified    |313140    |14.38           |
|debt_related |RENT          |Source Verified    |293559    |13.48           |
|debt_related |MORTGAGE      |Not Verified       |278355    |12.79           |
|debt_related |MORTGAGE      |Verified           |245576    |11.28           |
|debt_related |RENT          |Not Verified       |237859    |10.93           |
|debt_related |RENT          |Verified           |179809    |8.26            |
|debt_related |OWN           |Source Verified    |72746     |3.34            |
|debt_related |OWN           |Not Verified       |62577     |2.87            |
|large_expense|MORTGAGE      |Not Verified       |50956     |2.34            |
|large_expense|MORTGAGE  

In [39]:

# Umbral mínimo de registros para incluir una combinación en M
UMBRAL = 60000

# Lista de combinaciones válidas
combinaciones_validas = df_v1.groupBy("purpose", "home_ownership", "verification_status") \
    .count() \
    .filter(col("count") > UMBRAL) \
    .collect()

# Construimos la muestra M a partir de combinaciones válidas
muestras = []

for fila in combinaciones_validas:
    p, h, v = fila["purpose"], fila["home_ownership"], fila["verification_status"]
    mi = df_v1.filter((col("purpose") == p) & 
                   (col("home_ownership") == h) & 
                   (col("verification_status") == v))
    mi = mi.sample(withReplacement=False, fraction=0.2, seed=1)
    muestras.append(mi)

# Unión final para obtener M
M = muestras[0]
for mi in muestras[1:]:
    M = M.union(mi)

print("Total de registros en la muestra M:", M.count())

# Verifica distribución
M.groupBy("purpose", "home_ownership", "verification_status").count().orderBy("count", ascending=False).show(truncate=False)


Total de registros en la muestra M: 336754
+------------+--------------+-------------------+-----+
|purpose     |home_ownership|verification_status|count|
+------------+--------------+-------------------+-----+
|debt_related|MORTGAGE      |Source Verified    |62681|
|debt_related|RENT          |Source Verified    |58789|
|debt_related|MORTGAGE      |Not Verified       |55834|
|debt_related|MORTGAGE      |Verified           |49013|
|debt_related|RENT          |Not Verified       |47474|
|debt_related|RENT          |Verified           |35964|
|debt_related|OWN           |Source Verified    |14504|
|debt_related|OWN           |Not Verified       |12495|
+------------+--------------+-------------------+-----+



In [40]:
# Nueva columna binaria: riesgo = 1, No_riesgo  = 0
M = M.withColumn(
    "loan_status",
    F.when(F.col("loan_status") == "riesgo", 1).otherwise(0)
)

In [41]:
columnas_categoricas = [c for c, t in M.dtypes if t == "string" and c != "loan_status"]

indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_index", handleInvalid="keep")
    for c in columnas_categoricas
]
pipeline_indexado = Pipeline(stages=indexers)
M_r = pipeline_indexado.fit(M).transform(M)

In [42]:
M_r = M_r.drop(*columnas_categoricas)
M_r.show()

+---------+--------+------------------+------------------+-----------+------------------+------------------+------------------+------------------+----------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+------------------+-------+------------------+--------------------+---------------------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+----------------+------------------+------------------+--------------------------+-----------+----------------+--------------------+-------------------------+-------------+
|loan_amnt|int_rate|       installment|        annual_inc|loan_status|               dti|    inq_last_6mths|          open_acc|         revol_bal|revol_util|         total_acc|         out_prncp|       total_pymnt|   total_rec_prncp|     total_rec_int|   l

In [43]:
M_r.groupBy('loan_status').count().orderBy("count", ascending=False).show()

+-----------+------+
|loan_status| count|
+-----------+------+
|          0|293577|
|          1| 43177|
+-----------+------+



# **2. Construcción del conjunto Train – Test**

Una vez generada la muestra representativa `M` a partir de las particiones válidas `Mi` (seleccionadas con base en un umbral mínimo de 60,000 registros por combinación), se procede a dividir cada `Mi` en dos subconjuntos:

- `Tri`: conjunto de entrenamiento
- `Tsi`: conjunto de prueba

Con el objetivo de mantener la representatividad y evitar sesgos, se implementa una estrategia de muestreo **estratificado**, garantizando que:

- No exista traslape entre `Tri` y `Tsi` (es decir, `Tri ∩ Tsi = ∅`).
- La unión de todas las particiones (`Tri ∪ Tsi`) sea equivalente a la muestra total `M`.
- Se preserve la proporción original de cada combinación dentro de los conjuntos resultantes.

La proporción utilizada para la división es:

- **80% para el conjunto de entrenamiento (`Train`)**
- **20% para el conjunto de prueba (`Test`)**

A continuación se muestra el código correspondiente a este proceso.



In [44]:
# Porcentaje para el conjunto de entrenamiento
train_fraction = 0.8

# Obtener combinaciones únicas de Mi dentro de M
combinaciones_M = M_r.select("purpose_index", "home_ownership_index", "verification_status_index").distinct().collect()

# Listas para guardar subconjuntos
particiones_train = []
particiones_test = []

# Dividir cada Mi en Tri y Tsi
for fila in combinaciones_M:
    p, h, v = fila["purpose_index"], fila["home_ownership_index"], fila["verification_status_index"]
    
    # Extraer Mi
    Mi = M_r.filter(
        (col("purpose_index") == p) &
        (col("home_ownership_index") == h) &
        (col("verification_status_index") == v)
    )
    
    # División estratificada
    Tri, Tsi = Mi.randomSplit([train_fraction, 1 - train_fraction], seed=1)
    
    particiones_train.append(Tri)
    particiones_test.append(Tsi)

# Unir todas las particiones
Train = particiones_train[0]
for parte in particiones_train[1:]:
    Train = Train.union(parte)

Test = particiones_test[0]
for parte in particiones_test[1:]:
    Test = Test.union(parte)

# Verificar los resultados
print("Registros en Train:", Train.count())
print("Registros en Test:", Test.count())
print("Total original en M:", M_r.count())


Registros en Train: 269225
Registros en Test: 67529
Total original en M: 336754


# **3. Selección de métricas para medir calidad de resultados**

Antes de entrenar modelos de aprendizaje automático, es esencial definir las métricas que se utilizarán para evaluar la calidad de los resultados. Estas métricas deben ser tanto estadísticamente representativas como computacionalmente eficientes, especialmente considerando el entorno de Big Data y el uso de PySpark.

---

###  **Modelos supervisados: Clasificación**

Dado que la variable objetivo (`loan_status`) representa un problema de **clasificación binaria**, las métricas seleccionadas son:

- **Accuracy**: Proporción de predicciones correctas respecto al total. Es útil cuando las clases están balanceadas.
- **Precision**: Proporción de verdaderos positivos entre todas las predicciones positivas. Es fundamental cuando se desea minimizar falsos positivos.
- **Recall**: Proporción de verdaderos positivos entre todos los casos positivos reales. Es clave cuando se busca minimizar falsos negativos.
- **F1 Score**: Media armónica entre precisión y recall. Ideal para escenarios con clases desbalanceadas.

Estas métricas se implementarán utilizando los evaluadores nativos de PySpark:
- `MulticlassClassificationEvaluator`
- `BinaryClassificationEvaluator`

---

###  **Modelos no supervisados: Clustering**

En el caso de algoritmos como **K-Means**, donde no se dispone de etiquetas reales, la evaluación se basa en la calidad de los grupos formados. La métrica seleccionada es:

- **Silhouette Score**: Mide qué tan similares son los puntos dentro de un mismo clúster y qué tan distintos son respecto a otros clústeres. Una puntuación cercana a 1 indica agrupamientos bien definidos.

Esta métrica se calculará utilizando `ClusteringEvaluator` de PySpark, el cual está optimizado para operaciones distribuidas.

---

###  **Conclusión**

Las métricas seleccionadas permiten evaluar de manera integral la calidad de los modelos, tanto en tareas supervisadas como no supervisadas. En la etapa siguiente se implementarán estas métricas dentro de los experimentos de entrenamiento, facilitando así una comparación objetiva entre diferentes algoritmos y configuraciones en entornos de Big Data.


# **4. Entrenamiento de Modelos de Aprendizaje**

En esta sección se entrenan dos modelos de aprendizaje basados en los conjuntos `Train` y `Test` construidos previamente. Se abordan dos enfoques complementarios:

- **Aprendizaje supervisado**, orientado a predecir la variable binaria `loan_status`.
- **Aprendizaje no supervisado**, enfocado en identificar patrones de agrupamiento entre clientes con características similares.

Ambos modelos fueron elegidos por su **eficiencia computacional**, **escalabilidad** en ambientes distribuidos y su implementación nativa en PySpark.

---

### **4.1 Modelo Supervisado – Random Forest Classifier**

El objetivo principal del modelo supervisado es **predecir el nivel de riesgo crediticio (`loan_status`)**. El flujo de entrenamiento contempló los siguientes pasos:

- Codificación de variables categóricas mediante `StringIndexer`.
- Ensamblado de variables predictoras en un único vector mediante `VectorAssembler`.
- Entrenamiento del modelo usando `RandomForestClassifier` con configuración de `numTrees=50` y `maxDepth=5`.
- Evaluación sobre el conjunto de prueba (`Test`) con métricas como:
  - **Accuracy**
  - **Precision**
  - **Recall**
  - **F1 Score**
  
Para evitar sobreajuste, se aplicó validación basada en el conjunto `Test` y se limitaron hiperparámetros como la profundidad del árbol (`maxDepth`). Además, se empleó `seed` fijo para asegurar replicabilidad.

---

### **4.2 Modelo No Supervisado – KMeans Clustering**

Como complemento al modelo supervisado, se entrenó un modelo no supervisado de tipo **KMeans** para detectar patrones de agrupamiento entre clientes.

El procedimiento fue el siguiente:

- Ensamblado de estas variables en un vector de características (`features`).
- Entrenamiento del modelo KMeans con **k = 3** clusters iniciales.
- Evaluación de la calidad del agrupamiento utilizando **Silhouette Score** como métrica principal.

Se exploraron diferentes valores de `k` para observar la estabilidad de los clusters y se evaluó visualmente la separación entre grupos en función de variables críticas.

---

A continuación se presenta el código utilizado para la construcción, entrenamiento y evaluación de ambos modelos, junto con las métricas correspondientes.



In [45]:
# Separar clases
minority_class = Train.filter(F.col("loan_status") == 1)
majority_class = Train.filter(F.col("loan_status") == 0)
count_maj = majority_class.count()
count_min = minority_class.count()
balance_ratio = count_maj / count_min
# Oversample minoría (duplicar o triplicar)
oversampled_minority = minority_class.sample(withReplacement=True, fraction=balance_ratio, seed=1)

# Unir ambos
df_balanced = majority_class.union(oversampled_minority)

In [46]:
# Variables
target = 'loan_status'
features = [c for c in M_r.columns if c != target] 

# Ensamblar las variables en un vector de entrada
assembler = VectorAssembler(inputCols=features, outputCol="features")

# Modelo Random Forest
rf = RandomForestClassifier(labelCol=target, featuresCol="features", numTrees=50, maxDepth=5, seed=1)

# Pipeline
pipeline = Pipeline(stages=[assembler, rf])

# Entrenar modelo
model = pipeline.fit(df_balanced)

# Predicciones
predictions = model.transform(Test)



In [47]:
# Evaluación
evaluator_acc = MulticlassClassificationEvaluator(labelCol="loan_status", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="loan_status", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="loan_status", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="loan_status", predictionCol="prediction", metricName="weightedRecall")

print("Accuracy:", evaluator_acc.evaluate(predictions))
print("F1 Score:", evaluator_f1.evaluate(predictions))
print("Precision:", evaluator_precision.evaluate(predictions))
print("Recall:", evaluator_recall.evaluate(predictions))

Accuracy: 0.9083949118156643
F1 Score: 0.9149354855271792
Precision: 0.9288571956837178
Recall: 0.9083949118156643


In [48]:
conteo = predictions.groupBy("loan_status", "prediction").count()
conteo.orderBy(col("loan_status"), col("prediction")).show()


+-----------+----------+-----+
|loan_status|prediction|count|
+-----------+----------+-----+
|          0|       0.0|53971|
|          0|       1.0| 4923|
|          1|       0.0| 1263|
|          1|       1.0| 7372|
+-----------+----------+-----+



In [49]:
# Variables
variables_cluster  = [c for c in M_r.columns if c != target]
# Ensamblar vector de entrada para clustering
assembler = VectorAssembler(inputCols=variables_cluster, outputCol="features")
df_cluster = assembler.transform(df_balanced.select(*variables_cluster))

# Modelo KMeans con k=3
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=3, seed=1)
modelo_kmeans = kmeans.fit(df_cluster)

# Predicción de clusters
predicciones_cluster = modelo_kmeans.transform(df_cluster)


In [50]:

# Evaluación con Silhouette Score
evaluador_cluster = ClusteringEvaluator(featuresCol="features", predictionCol="cluster", metricName="silhouette")
silhouette = evaluador_cluster.evaluate(predicciones_cluster)

print(f"Silhouette Score para KMeans con k=3: {silhouette}")

Silhouette Score para KMeans con k=3: 0.7188812132520229


# **5. Análisis de Resultados**

Tras la implementación de los modelos de aprendizaje supervisado y no supervisado, se realizó un análisis detallado del desempeño de cada enfoque para evaluar su capacidad predictiva y de segmentación, respectivamente.

---

### **5.1 Modelo Supervisado – Random Forest**

El modelo de clasificación entrenado con Random Forest logró resultados sobresalientes tras aplicar balanceo de clases (oversampling de la clase minoritaria). Los resultados sobre el conjunto de prueba (`Test`) fueron:

- **Accuracy**: 0.9084  
- **F1 Score**: 0.9140  
- **Precision**: 0.9283  
- **Recall**: 0.9083  

La **matriz de confusión** fue la siguiente:

| loan_status | prediction | count |
|-------------|------------|-------|
| 0           | 0.0        | 53971 |
| 0           | 1.0        | 4923  |
| 1           | 0.0        | 1263  |
| 1           | 1.0        | 7372  |

Estos resultados reflejan una capacidad alta del modelo para **identificar correctamente tanto a los clientes que no caen en incumplimiento como a los que sí lo hacen**. El bajo número de falsos negativos (clase 1 predicha como 0) indica que el modelo es confiable para detectar riesgos crediticios, lo cual es crucial para mitigar pérdidas financieras. Además, se logró un equilibrio adecuado sin comprometer excesivamente la precisión.

---

### **5.2 Modelo No Supervisado – KMeans Clustering**

Se entrenó un modelo de agrupamiento KMeans con **k = 3**, utilizando un subconjunto de variables numéricas seleccionadas. La evaluación con **Silhouette Score** mostró:

- **Silhouette Score**: 0.7189

Este valor indica una **buena cohesión y separación** entre los grupos encontrados, lo que sugiere que existen patrones diferenciados en la población de clientes. Esto puede ser útil para **segmentación de mercados, análisis exploratorio o detección de perfiles** de comportamiento crediticio.

---

### **Conclusión General**

Ambos modelos demostraron desempeños efectivos:

- El modelo supervisado es altamente confiable para **predicciones directas** de riesgo crediticio.
- El modelo KMeans es útil para **descubrir patrones ocultos** sin depender de etiquetas, complementando el análisis supervisado.

Este enfoque dual puede ser integrado en soluciones más completas de análisis y monitoreo, combinando predicción de riesgo con segmentación estratégica de clientes.

